In [2]:
#  AI 美食顧問系統 + 訓練模型程式碼 + 圖片預覽功能

import os
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from PIL import Image
import pandas as pd
import matplotlib.pyplot as plt

# 設定裝置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 資料與模型路徑
food_info_path = "C:/Users/echo6/Downloads/Food/food_info.csv"
data_path = "C:/Users/echo6/Downloads/Food/data/food-101-tiny"
model_weights_path = "model_food.pth"

# 圖片轉換
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 載入資料集
train_dataset = datasets.ImageFolder(root=os.path.join(data_path, "train"), transform=transform)
valid_dataset = datasets.ImageFolder(root=os.path.join(data_path, "valid"), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

# 模型架構
class LeNet5(nn.Module):
    def __init__(self, num_classes):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.AvgPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 載入說明
df = pd.read_csv(food_info_path)
food_dict = dict(zip(df['food'], df['description']))
class_names = sorted(list(food_dict.keys()))

# 建立模型
num_classes = len(class_names)
model = LeNet5(num_classes).to(device)
if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path, map_location=device))
    print("✅ 模型已成功載入")
else:
    print("⚠️ 尚未載入模型權重，請確認是否先訓練模型")
model.eval()

def analyze_food_image(image_path):
    try:
        image = Image.open(image_path).convert('RGB')
        display(image)
    except Exception as e:
        print(f"❌ 圖片讀取錯誤: {e}")
        return

    input_tensor = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(input_tensor)
        probs = torch.softmax(output, dim=1)
        top3 = torch.topk(probs, 3)

        print("📊 預測前 3 名：")
        for i in range(3):
            idx = top3.indices[0][i].item()
            score = top3.values[0][i].item()
            print(f"Top {i+1}: {class_names[idx]}（信心值：{score:.2%}）")

        class_index = top3.indices[0][0].item()
        food_name = class_names[class_index]

    print(f"\n🍽️ 預測食物（最可能）：{food_name}")
    print(f"📘 食物說明：{food_dict.get(food_name, '查無資料')}")

# 🧠 訓練模型（如尚未訓練）
def train(model, loader):
    model.train()
    running_loss, correct, total = 0, 0, 0
    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
    return running_loss / len(loader), correct / total

def validate(model, loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
    return correct / total

# 執行訓練
optimizer = optim.Adam(model.parameters(), lr=0.001)
EPOCHS = 10
for epoch in range(EPOCHS):
    loss, train_acc = train(model, train_loader)
    val_acc = validate(model, valid_loader)
    print(f"Epoch {epoch+1}: Loss={loss:.4f}, Train Acc={train_acc:.4f}, Val Acc={val_acc:.4f}")

# 儲存模型
torch.save(model.state_dict(), model_weights_path)
print("✅ 模型訓練完成並已儲存為 model_food.pth")


✅ 模型已成功載入
Epoch 1: Loss=1.1726, Train Acc=0.5800, Val Acc=0.2800
Epoch 2: Loss=1.0364, Train Acc=0.6373, Val Acc=0.2840
Epoch 3: Loss=0.9902, Train Acc=0.6493, Val Acc=0.2840
Epoch 4: Loss=0.9456, Train Acc=0.6787, Val Acc=0.2920
Epoch 5: Loss=0.9065, Train Acc=0.6980, Val Acc=0.3000
Epoch 6: Loss=0.8650, Train Acc=0.7060, Val Acc=0.2980
Epoch 7: Loss=0.8330, Train Acc=0.7240, Val Acc=0.2820
Epoch 8: Loss=0.7891, Train Acc=0.7367, Val Acc=0.2760
Epoch 9: Loss=0.7183, Train Acc=0.7687, Val Acc=0.2860
Epoch 10: Loss=0.6710, Train Acc=0.7987, Val Acc=0.2840
✅ 模型訓練完成並已儲存為 model_food.pth
